In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.layers import Dense, Concatenate, Dot, Input, Lambda, BatchNormalization, Activation, Add
from tensorflow.keras.models import Sequential, Model
import keras_tuner
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fielding/player_fielding.csv


In [2]:
file=pd.read_csv('/kaggle/input/fielding/player_fielding.csv')

In [3]:
dct={}
for i in list(file):
    try:
        dct[type(file[i][51000])]+=1
    except:
        dct[type(file[i][51000])]=0
    

In [4]:
dct

{str: 2, numpy.float64: 10}

In [5]:
for i in list(file):
    if type(file[i][51000])==str:
        file.drop(i,axis=1, inplace=True)

In [6]:
len(list(file))

11

In [7]:
catches=file['Target_match_catches']

stumpings=file['Target_match_stumpings']

runouts=file['Target_match_runouts']


In [8]:
for i in list(file):
    if 'Target' in i:
        file.drop(i,axis=1, inplace=True)

In [9]:
file.fillna(0)

,match_id,previous_catches,previous_runouts,previous_stumpings,previous_innings_fielded,pFa_catches,pFa_stumpings,pFa_runouts
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...
53557,2409.0,0.0,0.0,0.0,2.0,0.000000,0.0,0.0
53558,2409.0,4.0,0.0,0.0,18.0,0.222222,0.0,0.0
53559,2409.0,1.0,0.0,0.0,8.0,0.125000,0.0,0.0
53560,2409.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0


In [10]:
groups=file.groupby('match_id')

In [11]:
# opposition_data=pd.DataFrame()
# for i in file['match_id'].unique():
#     grp=groups.get_group(i)
#     grp=grp.reset_index(drop=True)
#     grp=grp.drop('match_id',axis=1)
#     for j in grp.index:
#         drop_grp=grp.drop(j)
#         arr=drop_grp.to_numpy()
#         new_row_df = pd.DataFrame([arr.flatten()])

# # Concatenate the new row DataFrame with the original DataFrame
#         opposition_data = pd.concat([opposition_data, new_row_df], ignore_index=True)
#     if i%100==0:
#      print(i)
# opposition_data.fillna(0,inplace=True)

In [12]:
# import pandas as pd
# import numpy as np

# # Create an empty list to store processed data
# opposition_data_list = []

# # Iterate over each unique match_id
# for i, match_id in enumerate(file['match_id'].unique()):
#     grp = groups.get_group(match_id).reset_index(drop=True).drop(columns=['match_id'])

#     # Convert the DataFrame to a NumPy array once (faster operations)
#     grp_array = grp.to_numpy()

#     # Generate opposition data for each player
#     for j in range(len(grp_array)):
#         opposition_array = np.delete(grp_array, j, axis=0).flatten()
#         opposition_data_list.append(opposition_array)  # Store in list

#     # Print progress every 100 matches
#     if i % 100 == 0:
#         print(f"Processed {i} matches...")

# # Convert the final list to a DataFrame in one go (efficient)
# opposition_data = pd.DataFrame(opposition_data_list)

# opposition_data.fillna(0,inplace=True)

In [13]:
lst=[i for i in list(file) if ('v'==i[0] or 'v'==i[1]) ]
venue_features=file[lst]

In [14]:
file.drop(['match_id'],axis=1,inplace=True)

In [15]:
from keras.layers import Layer, Dense, BatchNormalization, Activation, Add, Lambda
import keras.backend as K

class BottleneckResidualBlock(Layer):
    def __init__(self, units=200):
        super(BottleneckResidualBlock, self).__init__()
        self.units = units
        self.dense1 = Dense(units)
        self.bn1 = BatchNormalization()
        self.activation1 = Activation('relu')
        self.dense2 = Dense(units)
        self.bn2 = BatchNormalization()
        self.activation2 = Activation('relu')
        self.dense3 = Dense(units)
        self.bn3 = BatchNormalization()
        self.activation3 = Activation('relu')
        self.dense4 = Dense(units)
        self.bn4 = BatchNormalization()
        self.activation4 = Activation('relu')

    def call(self, inputs):
        X_shortcut = inputs
        X = self.dense1(inputs)
        X = self.bn1(X)
        X = self.activation1(X)
        X = self.dense2(X)
        X = self.bn2(X)
        X = self.activation2(X)
        X = self.dense3(X)
        X = self.bn3(X)
        X = self.activation3(X)
        X = self.dense4(X)
        X = self.bn4(X)
        X = self.activation4(X)
        X = Add()([X, X_shortcut])
        return self.activation4(X)


In [16]:
def objective(hp):
    # num_opp=hp.Int("num_opp", 1, 100)
    num_venue=hp.Int("num_venue", 1, 50)
    num_self=hp.Int("num_self", 1, 30)
    num_net=hp.Int("num_net", 1, 15)
    
    # input_venue=Input(shape=(venue_player.loc[0]).shape)
    input_self=Input(shape=(file.loc[0]).shape)
    input_venue=Input(shape=(venue_features.loc[0]).shape)
    venue_nn=Sequential()
    venue_nn.add(Dense(units=200,activation='relu'))
    for  i in range(num_venue):
        if i<8:
            venue_nn.add(Dense(units=200,activation='relu'))
    
        else:
            if num_venue%4==0:
              venue_nn.add(BottleneckResidualBlock())
    venue_nn.add(Dense(units=100,activation='relu')) 
    venue_nn.add(Dense(units=25,activation='relu'))
    venue_nn.add(Dense(units=20,activation='relu'))
    # venue_nn=Sequential()
    # venue_nn.add(Dense(units=100,activation='relu'))
    # venue_nn.add(Dense(units=200,activation='relu'))
    # venue_nn.add(Dense(units=300,activation='relu'))
    # # venue_nn.add(Dense(units=1000))
    # for  i in range(num_venue):
    #     if i<8:
    #         venue_nn.add(Dense(units=300,activation='relu'))
    #     else:
    #         if num_venue%4==0:
    #             venue_nn.add(BottleneckResidualBlock())
    # # venue_nn.add(Dense(units=1000))
    # # venue_nn.add(Dense(units=400))
    # venue_nn.add(Dense(units=100,activation='relu')) 
    # venue_nn.add(Dense(units=25,activation='relu'))
    
    # venue_nn.add(Dense(units=20,activation='relu'))  
    self_nn=Sequential()
    self_nn.add(Dense(units=20,activation='relu'))
    # self_nn.add(Dense(units=1000))
    for  i in range(num_self):
        if i<20:
            self_nn.add(Dense(units=20,activation='relu'))
        else:
            if num_self%4==0:
                self_nn.add(BottleneckResidualBlock(units=20))
    # self_nn.add(Dense(units=1000))
    # self_nn.add(Dense(units=400))
    # self_nn.add(Dense(units=100,activation='relu')) 
    # self_nn.add(Dense(units=25,activation='relu'))
    self_nn.add(Dense(units=20,activation='relu'))    
    concatenated = Concatenate()([self_nn(input_self), venue_nn(input_venue)])
    circumstances_nn=Dense(units=40,activation='relu')(concatenated)
    circumstances_nn=Dense(units=40,activation='relu')(circumstances_nn)
    for  i in range(num_net):
        if i<8:
            circumstances_nn=Dense(units=40,activation='relu')(circumstances_nn)
        # else:
        #    if num_venue%4==0:
        #        circumstances_nn=BottleneckResidualBlock(units=40)(circumstances_nn)
    circumstances_nn=Dense(units=20,activation='relu')(circumstances_nn)            
    # output=Concatenate()([opp_nn(input_venue),self_nn(input_self)])
    # output=Dense(units=40,activation='relu')(output)
    # output=Dense(units=40,activation='relu')(output)
    # output=Dense(units=1,activation='relu')(output)
    model= Model([input_self,input_venue],circumstances_nn)
    model.compile(
        optimizer='adam',
        loss="mse",
        metrics=["mse",'mae'],
    )
   
    return model

In [17]:
file.replace([np.inf, -np.inf], 0, inplace=True)

In [18]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True
)

In [19]:
self_train = file.loc[:int(0.8*len(file))]
venue_train = venue_features.loc[:int(0.8*len(file))]
#opp_train = opposition_data.loc[:int(0.8*len(file))]
catches_train = catches.loc[:int(0.8*len(file))]
stumpings_train = stumpings.loc[:int(0.8*len(file))]
runouts_train = runouts.loc[:int(0.8*len(file))]

self_test = file.loc[int(0.8*len(file)):]
venue_test = venue_features.loc[int(0.8*len(file)):]
#opp_test = opposition_data.loc[int(0.8*len(file)):]
catches_test = catches.loc[int(0.8*len(file)):]
stumpings_test = stumpings.loc[int(0.8*len(file)):]
runouts_test = runouts.loc[int(0.8*len(file)):]

In [20]:
import tensorflow as tf
tf.device('/GPU:0')

In [21]:
# List all physical GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set the first GPU as the visible device
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Verify that the first GPU is being used
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"Number of Physical GPUs: {len(gpus)}, Number of Logical GPUs: {len(logical_gpus)}")

    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

Number of Physical GPUs: 1, Number of Logical GPUs: 1


In [22]:
# tuner1 = keras_tuner.RandomSearch(
#     hypermodel=objective,
#     objective="val_loss",
#     max_trials=20,
#     executions_per_trial=1,
#     overwrite=True
# )

# tuner1.search(
#     [self_train, venue_train], 
#     catches_train, 
#     epochs=20,
#     batch_size=128,
#     callbacks=[early_stopping], 
#     validation_data=([self_test, venue_test], catches_test)
# )

In [23]:
# tuner2 = keras_tuner.RandomSearch(
#     hypermodel=objective,
#     objective="val_loss",
#     max_trials=20,
#     executions_per_trial=1,
#     overwrite=True
# )

# tuner2.search(
#     [self_train, venue_train], 
#     stumpings_train, 
#     epochs=20,
#     batch_size=128,
#     callbacks=[early_stopping], 
#     validation_data=([self_test, venue_test], stumpings_test)
# )

In [24]:
# tuner3 = keras_tuner.RandomSearch(
#     hypermodel=objective,
#     objective="val_loss",
#     max_trials=20,
#     executions_per_trial=1,
#     overwrite=True
# )

# tuner3.search(
#     [self_train, venue_train], 
#     runouts_train, 
#     epochs=20,
#     batch_size=128,
#     callbacks=[early_stopping], 
#     validation_data=([self_test, venue_test], runouts_test)
# )

In [25]:
# def bottleneck_residual_block(X,  s=2):
#  X_shortcut=X
#  # Second component of main path
#  X =Dense(1000)(X)
#  X = BatchNormalization()(X)
#  X = Activation('relu')(X)
#  X =Dense(1000)(X)
#  X = BatchNormalization()(X)
#  X = Activation('relu')(X)
#  X =Dense(1000)(X)
#  X = BatchNormalization()(X)
#  X = Activation('relu')(X)
#  X =Dense(1000)(X)
#  X = BatchNormalization()(X)
#  X = Activation('relu')(X)
#  X = Add()([X, X_shortcut])
#  X = Activation('relu')(X)
#  return X

In [26]:

# def red_bottleneck_residual_block(X,  s=2):
#     X_shortcut=X
#     # Second component of main path
#     X =Dense(100)(X)
#     X = BatchNormalization()(X)
#     X = Activation('relu')(X)
#     X =Dense(100)(X)
#     X = BatchNormalization()(X)
#     X = Activation('relu')(X)
#     X =Dense(100)(X)
#     X = BatchNormalization()(X)
#     X = Activation('relu')(X)
#     X =Dense(100)(X)
#     X = BatchNormalization()(X)
#     X = Activation('relu')(X)
#     X = Add()([X, X_shortcut])
#     X = Activation('relu')(X)
#     return X

In [31]:
def get_model(target):
    if target == "catches":
        num_self, num_venue, num_net = 7,10,4
    elif target == "stumpings":
        num_self, num_venue, num_net = 2,11,1
    elif target == "runouts":
        num_self, num_venue, num_net = 1,45, 7
    else:
        raise ValueError

    # input_venue=Input(shape=(venue_player.loc[0]).shape)
    input_self=Input(shape=(file.loc[0]).shape)
    input_venue=Input(shape=(venue_features.loc[0]).shape)
    venue_nn=Sequential()
    venue_nn.add(Dense(units=200,activation='relu'))
    for  i in range(num_venue):
        if i<8:
            venue_nn.add(Dense(units=200,activation='relu'))
    
        else:
            if num_venue%4==0:
              venue_nn.add(BottleneckResidualBlock())
    venue_nn.add(Dense(units=100,activation='relu')) 
    venue_nn.add(Dense(units=25,activation='relu'))
    venue_nn.add(Dense(units=20,activation='relu'))
    # venue_nn=Sequential()
    # venue_nn.add(Dense(units=100,activation='relu'))
    # venue_nn.add(Dense(units=200,activation='relu'))
    # venue_nn.add(Dense(units=300,activation='relu'))
    # # venue_nn.add(Dense(units=1000))
    # for  i in range(num_venue):
    #     if i<8:
    #         venue_nn.add(Dense(units=300,activation='relu'))
    #     else:
    #         if num_venue%4==0:
    #             venue_nn.add(BottleneckResidualBlock())
    # # venue_nn.add(Dense(units=1000))
    # # venue_nn.add(Dense(units=400))
    # venue_nn.add(Dense(units=100,activation='relu')) 
    # venue_nn.add(Dense(units=25,activation='relu'))
    
    # venue_nn.add(Dense(units=20,activation='relu'))  
    self_nn=Sequential()
    self_nn.add(Dense(units=20,activation='relu'))
    # self_nn.add(Dense(units=1000))
    for  i in range(num_self):
        if i<20:
            self_nn.add(Dense(units=20,activation='relu'))
        else:
            if num_self%4==0:
                self_nn.add(BottleneckResidualBlock(units=20))
    # self_nn.add(Dense(units=1000))
    # self_nn.add(Dense(units=400))
    # self_nn.add(Dense(units=100,activation='relu')) 
    # self_nn.add(Dense(units=25,activation='relu'))
    self_nn.add(Dense(units=20,activation='relu'))    
    concatenated = Concatenate()([self_nn(input_self), venue_nn(input_venue)])
    circumstances_nn=Dense(units=40,activation='relu')(concatenated)
    circumstances_nn=Dense(units=40,activation='relu')(circumstances_nn)
    for  i in range(num_net):
        if i<8:
            circumstances_nn=Dense(units=40,activation='relu')(circumstances_nn)
        # else:
        #    if num_venue%4==0:
        #        circumstances_nn=BottleneckResidualBlock(units=40)(circumstances_nn)
    circumstances_nn=Dense(units=1,activation='relu')(circumstances_nn)            
    # output=Concatenate()([opp_nn(input_venue),self_nn(input_self)])
    # output=Dense(units=40,activation='relu')(output)
    # output=Dense(units=40,activation='relu')(output)
    # output=Dense(units=1,activation='relu')(output)
    model= Model([input_self,input_venue],circumstances_nn)
    model.compile(
        optimizer='adam',
        loss="mse",
        metrics=["mse",'mae'],
    )
   
    return model

In [33]:
from sklearn.metrics import mean_absolute_error

y_data = {'catches':(catches_train, catches_test),
'stumpings':(stumpings_train,stumpings_test),
'runouts':(runouts_train,runouts_test)}

for target in ('catches','stumpings','runouts'):
    model = get_model(target)
    model.fit([self_train, venue_train], y_data[target][0], batch_size=64, epochs=40, callbacks=[early_stopping], validation_data=([self_test, venue_test], y_data[target][1]))
    y_pred = model.predict([self_test, venue_test])
    print(f"{target} MAE: {mean_absolute_error(y_pred, y_data[target][1])}")
    model.save(f'/kaggle/working/{target}.keras')

Epoch 1/40
670/670 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - loss: 0.4402 - mae: 0.4921 - mse: 0.4402 - val_loss: 0.4195 - val_mae: 0.5150 - val_mse: 0.4195
Epoch 2/40
670/670 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.4047 - mae: 0.4866 - mse: 0.4047 - val_loss: 0.4161 - val_mae: 0.4922 - val_mse: 0.4161
Epoch 3/40
670/670 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.4105 - mae: 0.4865 - mse: 0.4105 - val_loss: 0.4170 - val_mae: 0.5010 - val_mse: 0.4170
Epoch 4/40
670/670 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.4071 - mae: 0.4823 - mse: 0.4071 - val_loss: 0.4113 - val_mae: 0.5006 - val_mse: 0.4113
Epoch 5/40
670/670 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.4109 - mae: 0.4837 - mse: 0.4109 - val_loss: 0.4113 - val_mae: 0.5092 - val_mse: 0.4113
Epoch 6/40
670/670 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.4232 - mae: 0.4937 - mse: 0.4232 - val_loss: 0.4133 - val_mae: 0.5090 - val_mse: 0.4133
Epoch 7/40
670/670 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.4093 - mae: 0.4833 - mse: 0.4093 - v

In [ ]:
y_data[target][1].shape